In [2]:
import pandas as pd
import numpy as np
import time
import math
import gc
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.image as mpimg
import seaborn as sns
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed , hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops , regionprops_table
from scipy import ndimage as ndi
import tifffile as tifi
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from datetime import datetime, timedelta
import openslide
from openslide import OpenSlide
import cv2 
import random
import copy
import pyarrow.parquet as pq
import pyarrow as pa
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, Callback
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB5

import warnings
warnings.filterwarnings('ignore')

In [3]:
def seed_everything(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(31)

In [4]:
train_df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')
test_df  = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')
train_df.head()

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


In [5]:
train_df["file_path"] = train_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai-png-train-files/train_images/" + x + ".png")
test_df["file_path"] = test_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/test/" + x + ".tif")
train_df["target"] = train_df["label"].apply(lambda x : '1' if x=="CE" else '0')
train_df.head()

,image_id,center_id,patient_id,image_num,label,file_path,target
0,006388_0,11,006388,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
1,008e5c_0,11,008e5c,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
2,00c058_0,11,00c058,0,LAA,../input/mayo-clinic-strip-ai-png-train-files/...,0
3,01adc5_0,11,01adc5,0,LAA,../input/mayo-clinic-strip-ai-png-train-files/...,0
4,026c97_0,4,026c97,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1


In [6]:
test_df.head()

,image_id,center_id,patient_id,image_num,file_path
0,006388_0,11,006388,0,../input/mayo-clinic-strip-ai/test/006388_0.tif
1,008e5c_0,11,008e5c,0,../input/mayo-clinic-strip-ai/test/008e5c_0.tif
2,00c058_0,11,00c058,0,../input/mayo-clinic-strip-ai/test/00c058_0.tif
3,01adc5_0,11,01adc5,0,../input/mayo-clinic-strip-ai/test/01adc5_0.tif


In [7]:
train_df.shape

(754, 7)

In [8]:
train,test = train_test_split(train_df,test_size=0.2)
train.head()

,image_id,center_id,patient_id,image_num,label,file_path,target
393,83198b_0,9,83198b,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
311,63f341_0,11,63f341,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
165,369366_0,3,369366,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
105,23d2c1_0,7,23d2c1,0,CE,../input/mayo-clinic-strip-ai-png-train-files/...,1
10,03d1ec_0,11,03d1ec,0,LAA,../input/mayo-clinic-strip-ai-png-train-files/...,0


In [9]:
train.shape, test.shape

((603, 7), (151, 7))

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=10,  # rotation
    width_shift_range=0.2,  # horizontal shift
    height_shift_range=0.2,  # vertical shift
    horizontal_flip=True,  # horizontal flip
    brightness_range=[0.2, 1.2],
)  # brightness)

train_gen = datagen.flow_from_dataframe(
    train,
    #directory = '../input/jpg-images-strip-ai/train',
    x_col='file_path', 
    y_col='target', 
    target_size=(512, 512), 
    color_mode='rgb',
    class_mode='binary', 
    batch_size=32,
    seed = 31,
)

test_gen = datagen.flow_from_dataframe(
    test,
    #directory = '../input/jpg-images-strip-ai/train',
    x_col='file_path', 
    y_col='target', 
    target_size=(512, 512), 
    color_mode='rgb',
    class_mode='binary', 
    batch_size=1,
    shuffle = False
)

Found 601 validated image filenames belonging to 2 classes.
Found 151 validated image filenames belonging to 2 classes.


In [11]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)
earstop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3)

In [21]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def model_MobileNetV2(lr=0.001, dr_rate=0.15):
    model = MobileNetV2(include_top=False, weights='imagenet')
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x = BatchNormalization()(x)
    x = Dropout(dr_rate)(x)
    dense_1 = Dense(64, activation="relu")(x)
    dense_2 = Dense(32, activation="relu")(dense_1)
    outputs = Dense(1, activation="sigmoid")(dense_2)

    # Compile
    model = Model(model.inputs, outputs, name="MobileNetV2")
    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"]
    )
    return model


In [26]:
model_MobileNetV2 = model_MobileNetV2()
model_MobileNetV2.summary()

Model: "MobileNetV2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, None, None, 3 0           bn_Conv1[0][0]                   
________________________________________________________________________________________

In [23]:
history_0 = model_MobileNetV2.fit(
    train_gen,
    epochs = 5,
    batch_size=32,
    validation_data = test_gen,
    verbose = 1,
    callbacks = [lrate, earstop]
)

19/19 [==============================] - 56s 3s/step - loss: 0.6254 - binary_accuracy: 0.7022 - val_loss: 0.6577 - val_binary_accuracy: 0.6755


In [24]:
# Find prediction for test_gen
from sklearn.metrics import accuracy_score
preds = model_MobileNetV2.predict(test_gen)

In [25]:
accuracy_score(test['target'].astype(int), np.round(preds).astype(int))

0.6622516556291391